Trying with Pyviz

Link to the Tutorial: [PyVis](https://pyvis.readthedocs.io/en/latest/tutorial.html)

Some Acronyms:
IG: Interactive Graph
SP: Scatter Plot

In [27]:
!pip install pyvis==0.3.1
!pip install plotly
from pyvis.network import Network
import pandas as pd
from pandas import DataFrame
import plotly.express as px
import textwrap

def extract_numbers(input_string):
    numbers_only = ''.join(char for char in input_string if char.isdigit())
    result_integer = int(numbers_only)
    return result_integer

def wrap_text(text, width=50):
    wrapper = textwrap.TextWrapper(width=width, break_long_words=False)
    return '\n'.join(wrapper.wrap(text))

def wrap_text_withBR(text, width=40):
    wrapper = textwrap.TextWrapper(width=width, break_long_words=False)
    return '<br>'.join(wrapper.wrap(text))

# def CenteredAlign(text, width=50):
#     wrapper = textwrap.TextWrapper(width=width, break_long_words=False)
#     return '<br>'.join(wrapper.wrap(text))

In [28]:
def processing(df):
  for column in df.columns:
    df[column] = df[column].str.lstrip().str.rstrip()
    return df

In [29]:
MainSheet = pd.read_excel("dt.xlsx", sheet_name='main', skiprows=0)
FeatureDescriptionSheet = pd.read_excel("dt.xlsx", sheet_name='FeatureDescription', skiprows=0)
ClassDescriptionSheet = pd.read_excel("dt.xlsx", sheet_name='ClassDescription', skiprows=0)
EntityDescriptionSheet = pd.read_excel("dt.xlsx", sheet_name='EntityDescription', skiprows=0)

MainSheet = MainSheet.drop(columns=['Article Title', 'Link', 'SL'])

In [30]:
MainSheet = processing(MainSheet)
FeatureDescriptionSheet = processing(FeatureDescriptionSheet)
ClassDescriptionSheet = processing(ClassDescriptionSheet)
EntityDescriptionSheet = processing(EntityDescriptionSheet)
# Display the modified DataFrame

In [31]:
MainSheet['ReACT_title'] = MainSheet['ReACT_title'].apply(wrap_text)

ReAct_Dict = dict(zip(MainSheet['ReACT_number'], MainSheet['ReACT_title']))
FeatureDescription_Dict = dict(zip(FeatureDescriptionSheet['FeatureName'], FeatureDescriptionSheet['DescriptionOfTheFeature']))
ClassDescription_Dict = dict(zip(ClassDescriptionSheet['Class'], ClassDescriptionSheet['ClassDescription']))
EntityDescription_Dict = dict(zip(EntityDescriptionSheet['Entity'], EntityDescriptionSheet['EntityDescription']))
MainSheet.head()

,ReACT_number,ReACT_title,Class,Entity_responsible,Features,Quatative/Qualatitative,References
0,ReACT-1,Maintaining a set of active/friendly developers,"Individual, Group","Project Maintainers, Project Leaders",num_act_dev,NaN,No. of reference: 2\n
1,ReACT-2,"Include the new members in the project, in whi...","Individual, Group","Project Maintainers, Project Leaders",num_act_dev,NaN,No. of reference: 1
2,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",Code,"Project Maintainers, Project Leaders","num_act_dev, top_e_fract, top_c_fract",NaN,No. of reference: 3\n
3,ReACT-4,Use most common SE principles,Code,"Project Maintainers, Project Leaders",num_act_dev,NaN,No. of reference: 1
4,ReACT-5,Doing Pull based development,Code,Developers,num_act_dev,NaN,No. of reference: 1


In [32]:
ReActAndTitle = MainSheet[['ReACT_number', 'ReACT_title']]

#Plotting ReAct and Class - Network Diagram

In [33]:
ReActAndClass = MainSheet[['ReACT_number', 'Class']]

b = DataFrame(ReActAndClass.Class.str.split(',').tolist(), index=ReActAndClass.ReACT_number).stack()
b = b.reset_index()[[0, 'ReACT_number']]
b.columns = ['Class', 'ReACT_number'] # renaming var1
ReActAndClass = b
ReActAndClass = processing(ReActAndClass)

In [34]:
ReActTitleClass = pd.merge(ReActAndTitle, ReActAndClass, on='ReACT_number', how='outer')
ReActTitleClass['ClassDescription'] = None

ReActClassDescription = ReActTitleClass
ReActClassDescription
for i in range(0,len(ReActClassDescription)):
  # print(ReActClassDescription.Class[i].lstrip().rstrip())
  # print(i, ClassDescription_Dict[ReActClassDescription.Class[i].lstrip().rstrip()])
  ReActClassDescription.ClassDescription[i] = ClassDescription_Dict[ReActClassDescription.Class[i].lstrip().rstrip()]
ReAct_Title_Class_ClassDescription = ReActClassDescription


In [35]:
ReACT_number = ReAct_Title_Class_ClassDescription['ReACT_number'].values
ReACT_Title = ReAct_Title_Class_ClassDescription['ReACT_title'].values
ReAct_Title_Class_ClassDescription['ReACT_title'] = ReAct_Title_Class_ClassDescription['ReACT_title'].apply(wrap_text)

Class = ReAct_Title_Class_ClassDescription['Class'].values
ReAct_Title_Class_ClassDescription['ClassDescription'] =  ReAct_Title_Class_ClassDescription['ClassDescription'].apply(wrap_text)
ClassDescription = ReAct_Title_Class_ClassDescription['ClassDescription'].values

net = Network(directed =True)
for i in range(0,len(ReACT_number)):
  # print(actionable[i])
  net.add_node(ReACT_number[i], label=ReACT_number[i], title=ReACT_Title[i])
  net.add_node(Class[i], label=Class[i], title=ClassDescription[i], color = "#5DBB63",shape='box')
  net.add_edge(ReACT_number[i], Class[i], color = "#016064")

# net.show_buttons(filter_=['physics'])
net.show('IG_Class.html')

In [36]:
ReAct_Title_Class_ClassDescription['ReACT_title'][0]

'Maintaining a set of active/friendly developers'

##BarPlot

In [37]:
unique_counts = ReAct_Title_Class_ClassDescription['Class'].value_counts().reset_index()

# Rename the columns for clarity
unique_counts.columns = ['Class of ReACTs', 'Count of ReACTs']

# Define an array of colors
darker_shades = ['darkred', 'lightblue', 'lightgreen', '#FFABAB', '#91a8a8', '#95ab8a', '#00a86b', '#8E44AD', '#2E4053', '#D35400', '#a37b65', '#548569', '#474466', '#FDEBD0', '#F9EBB2', '#EAECEE', '#8B5A80']

fig = px.bar(unique_counts, x='Class of ReACTs', y='Count of ReACTs', color='Class of ReACTs', color_discrete_sequence=darker_shades)

# Save the plot as an HTML file
fig.write_html('BarPlot_Class.html')
fig.show()

In [38]:
ReAct_Title_Class_ClassDescription

,ReACT_number,ReACT_title,Class,ClassDescription
0,ReACT-1,Maintaining a set of active/friendly developers,Individual,The Individual class emphasizes the role of\ni...
1,ReACT-1,Maintaining a set of active/friendly developers,Group,Activities that can be performed by working gr...
2,ReACT-2,"Include the new members in the project, in whi...",Individual,The Individual class emphasizes the role of\ni...
3,ReACT-2,"Include the new members in the project, in whi...",Group,Activities that can be performed by working gr...
4,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",Code,The Code class is centered around the quality ...
...,...,...,...,...
60,ReACT-47,"Have the project owned by users, rather than\n...",Project,The Project class focuses on the overall\nmana...
61,ReACT-47,"Have the project owned by users, rather than\n...",Individual,The Individual class emphasizes the role of\ni...
62,ReACT-48,Plan to finish the project in a shorter amount...,Project,The Project class focuses on the overall\nmana...
63,ReACT-49,Incorporating inclusive pronouns in communicat...,Individual,The Individual class emphasizes the role of\ni...


In [39]:
import plotly.graph_objects as go
import plotly.io as pio

subject = ReAct_Title_Class_ClassDescription['Class']
score = ReAct_Title_Class_ClassDescription['ReACT_number']

TextToShow = "<b>"+ReAct_Title_Class_ClassDescription['ReACT_number'] + " Belongs to </b>"
TextToShow += "<b>"+ReAct_Title_Class_ClassDescription['Class'] +"</b> Class"+ "<br>"
TextToShow += "<b>"+ReAct_Title_Class_ClassDescription['ReACT_number']+"</b>: "
TextToShow += ReAct_Title_Class_ClassDescription['ReACT_title'].apply(wrap_text_withBR)
TextToShow += "<br><b>" +ReAct_Title_Class_ClassDescription['Class'] +": </b>"
TextToShow += ReAct_Title_Class_ClassDescription['ClassDescription'].apply(wrap_text_withBR)


data = [dict(
  type = 'scatter',
  x = subject,
  y = score,
  mode = 'markers',
  text = TextToShow,  # Set the 'text' property to the Y-axis values
  transforms = [dict(
    type = 'groupby',
    groups = subject
  )],
   marker=dict(
        symbol='square',  # Set the shape of the marker (e.g., 'circle', 'square', 'diamond', etc.)
        size=10  # Set the size of the circles (adjust as needed)

    )
)]

layout = dict(xaxis=dict(title='Features'),
              yaxis=dict(title='ReACT Number'),
              height=1200,
              width=800)

fig_dict = dict(data=data, layout=layout)
pio.show(fig_dict, validate=False)
pio.write_html(fig_dict, file='SP_Class.html', auto_open=True, validate=False)

#Plotting ReAct and Entity - Network Diagram

In [40]:
ReActAndEntity = MainSheet[['ReACT_number', 'Entity_responsible']]

b = DataFrame(ReActAndEntity.Entity_responsible.str.split(',').tolist(), index=ReActAndEntity.ReACT_number).stack()
b = b.reset_index()[[0, 'ReACT_number']]
b.columns = ['Entity', 'ReACT_number'] # renaming var1
ReActAndEntity = b
ReActAndEntity = processing(ReActAndEntity) # performing lstrip rstrip after the separation
# ReActAndEntity.Entity.value_counts()

###Unique values in the Entity Column
(You neeed description for each item in the EntityDescription Sheet)
I wrote this to determine the unique values available in the Entity Column. So that, later, I can define each of them in the EntityDescription sheet

In [41]:
unique_values = set(ReActAndEntity.Entity)

# Convert the unique values back to a list if needed
unique_values_list = list(unique_values)

print(unique_values_list)
for i in range(0,len(unique_values_list)):
  print(unique_values_list[i].lstrip().rstrip())

['Developers', 'Documentation Team', 'Project Leaders', 'Sponsors', 'Community Moderators', 'Core Developers', 'Peer Reviewers', 'Technical Leads', 'Project Maintainers', 'Community Managers', 'Design Team', 'Collaboration Coordinators', 'Owners', 'Contributors', 'Quality Assurance Team', 'Mentors', 'Project Managers', 'Architects']
Developers
Documentation Team
Project Leaders
Sponsors
Community Moderators
Core Developers
Peer Reviewers
Technical Leads
Project Maintainers
Community Managers
Design Team
Collaboration Coordinators
Owners
Contributors
Quality Assurance Team
Mentors
Project Managers
Architects


In [42]:
ReActTitleEntity = pd.merge(ReActAndTitle, ReActAndEntity, on='ReACT_number', how='outer')
ReActTitleEntity['EntityDescription'] = None

ReActEntityDescription = ReActTitleEntity

for i in range(0,len(ReActEntityDescription)):
  ReActEntityDescription.EntityDescription[i] = EntityDescription_Dict[ReActEntityDescription.Entity[i].lstrip().rstrip()]
ReAct_Title_Entity_EntityDescription = ReActEntityDescription
ReAct_Title_Entity_EntityDescription

,ReACT_number,ReACT_title,Entity,EntityDescription
0,ReACT-1,Maintaining a set of active/friendly developers,Project Maintainers,Project maintainers in open-source software de...
1,ReACT-1,Maintaining a set of active/friendly developers,Project Leaders,Project leaders in open-source software develo...
2,ReACT-2,"Include the new members in the project, in whi...",Project Maintainers,Project maintainers in open-source software de...
3,ReACT-2,"Include the new members in the project, in whi...",Project Leaders,Project leaders in open-source software develo...
4,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",Project Maintainers,Project maintainers in open-source software de...
...,...,...,...,...
107,ReACT-48,Plan to finish the project in a shorter amount...,Project Leaders,Project leaders in open-source software develo...
108,ReACT-48,Plan to finish the project in a shorter amount...,Developers,"In open-source software development, developer..."
109,ReACT-49,Incorporating inclusive pronouns in communicat...,Contributors,Contributors in open-source software developme...
110,ReACT-49,Incorporating inclusive pronouns in communicat...,Community Managers,Community managers in open source oversee posi...


In [43]:
df = ReAct_Title_Entity_EntityDescription
ReACT_number = df['ReACT_number'].values

ReACT_Title = df['ReACT_title'].values

Entity = df['Entity'].values
df['EntityDescription'] = df['EntityDescription'].apply(wrap_text)
EntityDescription = df['EntityDescription'].values


net = Network(directed =True)
for i in range(0,len(ReACT_number)):
  # print(actionable[i])
  net.add_node(ReACT_number[i], label=ReACT_number[i], title=ReACT_Title[i])
  net.add_node(Entity[i], label=Entity[i], title=EntityDescription[i], color = "#98BF64",shape='box')
  net.add_edge(Entity[i], ReACT_number[i], color = "#32612D")

# net.show_buttons(filter_=['physics'])
net.show('IG_Entity.html')

In [54]:
unique_counts = ReAct_Title_Entity_EntityDescription['Entity'].value_counts().reset_index()

# Rename the columns for clarity
unique_counts.columns = ['Entity Responsible for Performing ReACTs', 'Count of ReACTs']

# Define an array of colors
darker_shades = ['darkred', 'lightblue', 'lightgreen', '#FFABAB', '#91a8a8',
                 '#95ab8a', '#00a86b', '#8E44AD', '#2E4053', '#D35400', '#a37b65',
                 '#6C5B7B', '#C0C999', '#7D8CC4', '#FFA07A', '#88D8B0',
    '#826c7F', '#D8D8D8', '#5F758E', '#E2B464', '#649E7D',
    '#CAAE77', '#7A9E9F', '#FAE5D3', '#BAC9FF', '#E47878',
    '#D2F2D2', '#7C83FD', '#F5A623', '#A77B7B', '#D8D8D8']


# Create an interactive bar plot using Plotly Express
fig = px.bar(unique_counts, x='Count of ReACTs', y='Entity Responsible for Performing ReACTs', color='Entity Responsible for Performing ReACTs', color_discrete_sequence=darker_shades, orientation='h')

# Save the plot as an HTML file
fig.write_html('BarPlot_Entity.html')
fig.show()

In [45]:
import plotly.graph_objects as go
import plotly.io as pio

subject = ReAct_Title_Entity_EntityDescription['Entity']
score = ReAct_Title_Entity_EntityDescription['ReACT_number']

TextToShow = "<b>"+ReAct_Title_Entity_EntityDescription['Entity'] + " are Responsible for Performing </b>"
TextToShow += "<b>"+ReAct_Title_Entity_EntityDescription['ReACT_number'] +"</b>"+ "<br>"
TextToShow += "<b>"+ReAct_Title_Entity_EntityDescription['ReACT_number']+"</b>: "
TextToShow += ReAct_Title_Entity_EntityDescription['ReACT_title'].apply(wrap_text_withBR)
TextToShow += "<br><b>" +ReAct_Title_Entity_EntityDescription['Entity'] +": </b>"
TextToShow += ReAct_Title_Entity_EntityDescription['EntityDescription'].apply(wrap_text_withBR)


data = [dict(
  type = 'scatter',
  x = subject,
  y = score,
  mode = 'markers',
  text = TextToShow,  # Set the 'text' property to the Y-axis values
  transforms = [dict(
    type = 'groupby',
    groups = subject
  )],
   marker=dict(
        symbol='square',  # Set the shape of the marker (e.g., 'circle', 'square', 'diamond', etc.)
        size=10  # Set the size of the circles (adjust as needed)

    )
)]

layout = dict(xaxis=dict(title='Features'),
              yaxis=dict(title='ReACT Number'),
              height=1200,
              width=1100)

fig_dict = dict(data=data, layout=layout)
pio.show(fig_dict, validate=False)
pio.write_html(fig_dict, file='SP_Entity.html', auto_open=True, validate=False)

#Plotting ReAct and Features - Network Diagram

In [46]:
ReActAndFeature = MainSheet[['ReACT_number', 'Features']]

b = DataFrame(ReActAndFeature.Features.str.split(',').tolist(), index=ReActAndFeature.ReACT_number).stack()
b = b.reset_index()[[0, 'ReACT_number']]
b.columns = ['Features', 'ReACT_number'] # renaming var1
ReActAndFeature  = b
ReActAndFeature = processing(ReActAndFeature)
ReActAndFeature

,Features,ReACT_number
0,num_act_dev,ReACT-1
1,num_act_dev,ReACT-2
2,num_act_dev,ReACT-3
3,top_e_fract,ReACT-3
4,top_c_fract,ReACT-3
...,...,...
75,top_e_fract,ReACT-47
76,top_c_fract,ReACT-47
77,top_e_fract,ReACT-48
78,top_c_fract,ReACT-48


In [47]:
ReActTitleFeature = pd.merge(ReActAndTitle, ReActAndFeature, on='ReACT_number', how='outer')
ReActTitleFeature['FeatureDescription'] = None
ReActAndReference = MainSheet[['ReACT_number', 'References']]

ReActFeatureDescription = ReActTitleFeature


for i in range(0,len(ReActFeatureDescription)):
 ReActFeatureDescription.FeatureDescription[i] = FeatureDescription_Dict[ReActFeatureDescription.Features[i].lstrip().rstrip()]
ReAct_Title_Feature_FeatureDescription = ReActFeatureDescription
ReAct_Title_Feature_FeatureDescription_Reference = pd.merge(ReAct_Title_Feature_FeatureDescription, ReActAndReference, on='ReACT_number', how='outer')
ReAct_Title_Feature_FeatureDescription_Reference

,ReACT_number,ReACT_title,Features,FeatureDescription,References
0,ReACT-1,Maintaining a set of active/friendly developers,num_act_dev,The number of Active Developers is the count o...,No. of reference: 2\n
1,ReACT-2,"Include the new members in the project, in whi...",num_act_dev,The number of Active Developers is the count o...,No. of reference: 1
2,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",num_act_dev,The number of Active Developers is the count o...,No. of reference: 3\n
3,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",top_e_fract,The percentages of emails performed by the top...,No. of reference: 3\n
4,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",top_c_fract,The percentages of Commits performed by the to...,No. of reference: 3\n
...,...,...,...,...,...
75,ReACT-47,"Have the project owned by users, rather than\n...",top_e_fract,The percentages of emails performed by the top...,No. of reference: 1
76,ReACT-47,"Have the project owned by users, rather than\n...",top_c_fract,The percentages of Commits performed by the to...,No. of reference: 1
77,ReACT-48,Plan to finish the project in a shorter amount...,top_e_fract,The percentages of emails performed by the top...,No. of reference: 1
78,ReACT-48,Plan to finish the project in a shorter amount...,top_c_fract,The percentages of Commits performed by the to...,No. of reference: 1


In [48]:
df = ReAct_Title_Feature_FeatureDescription_Reference
ReACT_number = df['ReACT_number'].values
ReACT_Title = df['ReACT_title'].values
Feature = df['Features'].values
df['FeatureDescription'] = df['FeatureDescription'].apply(wrap_text)
FeatureDescription = df['FeatureDescription'].values

Edges = ReAct_Title_Feature_FeatureDescription_Reference['References'].values



net = Network(directed =True)
for i in range(0,len(ReACT_number)):
 # print(actionable[i])
 net.add_node(ReACT_number[i], label=ReACT_number[i], title=ReACT_Title[i])
 net.add_node(Feature[i], label=Feature[i], title=FeatureDescription[i], color = "#E07772",shape='box')
 net.add_edge(ReACT_number[i], Feature[i], weight=extract_numbers(Edges[i]), title=Edges[i], color = "#898980")

#  net.add_edge(ReACT_number[i], Feature[i], weight=extract_numbers(Edges[i]), title=Edges[i], color = "#898980")

# net.show_buttons(filter_=['physics'])
net.show('IG_Feature.html')

In [49]:
ReAct_Title_Feature_FeatureDescription_Reference['Features'].value_counts()

num_act_dev    36
top_c_fract    10
c_edges        10
c_c_coef       10
c_long_tail     9
top_e_fract     4
num_emails      1
Name: Features, dtype: int64

In [53]:
unique_counts = ReAct_Title_Feature_FeatureDescription_Reference['Features'].value_counts().reset_index()

# Rename the columns for clarity
unique_counts.columns = ['Feature Name', 'Count']

# Define an array of colors
darker_shades = ['darkred', 'lightblue', 'lightgreen', '#FFABAB', '#91a8a8',
                 '#95ab8a', '#00a86b', '#8E44AD', '#2E4053', '#D35400', '#a37b65',
                 '#6C5B7B', '#C0C999', '#7D8CC4', '#FFA07A', '#88D8B0',
    '#826c7F', '#D8D8D8', '#5F758E', '#E2B464', '#649E7D',
    '#CAAE77', '#7A9E9F', '#FAE5D3', '#BAC9FF', '#E47878',
    '#D2F2D2', '#7C83FD', '#F5A623', '#A77B7B', '#D8D8D8']


# Create an interactive bar plot using Plotly Express
fig = px.pie(unique_counts, names='Feature Name', values='Count', color='Feature Name', color_discrete_sequence=darker_shades)

# Save the plot as an HTML file
fig.write_html('PiePlot_Feature.html')
fig.show()

In [51]:
ReAct_Title_Feature_FeatureDescription_Reference

,ReACT_number,ReACT_title,Features,FeatureDescription,References
0,ReACT-1,Maintaining a set of active/friendly developers,num_act_dev,The number of Active Developers is the count o...,No. of reference: 2\n
1,ReACT-2,"Include the new members in the project, in whi...",num_act_dev,The number of Active Developers is the count o...,No. of reference: 1
2,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",num_act_dev,The number of Active Developers is the count o...,No. of reference: 3\n
3,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",top_e_fract,The percentages of emails performed by the top...,No. of reference: 3\n
4,ReACT-3,"Use common programming language (PHP, JAVA,\nJ...",top_c_fract,The percentages of Commits performed by the to...,No. of reference: 3\n
...,...,...,...,...,...
75,ReACT-47,"Have the project owned by users, rather than\n...",top_e_fract,The percentages of emails performed by the top...,No. of reference: 1
76,ReACT-47,"Have the project owned by users, rather than\n...",top_c_fract,The percentages of Commits performed by the to...,No. of reference: 1
77,ReACT-48,Plan to finish the project in a shorter amount...,top_e_fract,The percentages of emails performed by the top...,No. of reference: 1
78,ReACT-48,Plan to finish the project in a shorter amount...,top_c_fract,The percentages of Commits performed by the to...,No. of reference: 1


In [52]:
import plotly.graph_objects as go
import plotly.io as pio



subject = ReAct_Title_Feature_FeatureDescription_Reference['Features']
score = ReAct_Title_Feature_FeatureDescription_Reference['ReACT_number']


TextToShow = ReAct_Title_Feature_FeatureDescription_Reference['ReACT_number'] + " Positively Impacts "
TextToShow += ReAct_Title_Feature_FeatureDescription_Reference['Features'] + "<br>"
TextToShow +=  "<b>"+ReAct_Title_Feature_FeatureDescription_Reference['ReACT_number']+"</b>: "
TextToShow += ReAct_Title_Feature_FeatureDescription_Reference['ReACT_title'].apply(wrap_text_withBR)
TextToShow += "<br><b>" +ReAct_Title_Feature_FeatureDescription_Reference['Features'] +": </b>"
TextToShow += ReAct_Title_Feature_FeatureDescription_Reference['FeatureDescription'].apply(wrap_text_withBR)

data = [dict(
  type = 'scatter',
  x = subject,
  y = score,
  mode = 'markers',
  text = TextToShow,
  transforms = [dict(
    type = 'groupby',
    groups = subject
  )],
   marker=dict(
        symbol='square',  # Set the shape of the marker (e.g., 'circle', 'square', 'diamond', etc.)
        size=10  # Set the size of the circles (adjust as needed)

    )
)]

layout = dict(xaxis=dict(title='Features'),
              yaxis=dict(title='ReACT Number'),
              height=1200,
              width=800)

fig_dict = dict(data=data, layout=layout)
pio.show(fig_dict, validate=False)
pio.write_html(fig_dict, file='SP_Feature.html', auto_open=True, validate=False)